# Ch13. 기계 번역 (Neural Machine Translation)

# v03. BLEU Score (Bilingual Evaluation Understudy Score)

- 앞서 언어 모델(Language Model)의 성능 측정을 위한 평가 방법(evaluation metric)으로 펄플렉서티(perplexity, PPL)를 소개한 바 있다.
- 기계 번역기에도 PPL을 평가에 사용할 수 있지만, PPL은 번역의 성능을 직접적으로 반영하는 수치라 보기엔 어렵다.

- 자연어 처리에서는 그 외에도 수많은 평가 방법들이 존재한다.
- 이번 챕터에서는 기계 번역의 성능이 얼마나 뛰어난 가를 측정하기 위해 사용되는 대표적인 방법인 BLEU(Bilingual Evaluation Understudy)에 대해서 배워본다.
- 이하 BLEU라고 명명한다.
- 이번 챕터에서는 BLEU의 개념에 대해서 이해하고, 가장 기본적인 BLEU를 구현해본다.

- 이번 챕터는 논문 BLEU: a Method for Automatic Evaluation of Machine Translation를 참고로 하여 작성되었다.

- 또한 아래의 실습은 아래의 코드가 먼저 임포트되었다고 가정한다.

In [0]:
from collections import Counter
import numpy as np
from nltk import ngrams

<br>

## 3.1 BLEU (Bilingual Evaluation Understudy)

- BLEU는 기계 번역 결과와 사람이 직접 번역한 결과가 얼마나 유사한 지 비교하여 번역에 대한 성능을 측정하는 방법이다.
- 측정 기준은 n-gram에 기반한다.
- n-gram의 정의는 "n-gram 언어 모델" 챕터를 참고하면 된다.

<br>

- BLEU는 완벽한 방법이라고는 할 수는 없지만 몇 가지 이점을 가진다.
  - 언어에 구애받지 않고 사용할 수 있다.
  - 계산 속도가 빠르다.
- BLEU는 PPL과는 달리 높을 수록 성능이 더 좋음을 의미한다.
- BLEU를 이해하기 위해 기계 번역 성능 평가를 위한 몇 가지 직관적인 방법을 먼저 제시하고, 문제점을 보완해나가는 방식으로 설명한다.

<br>

### 3.1.1 유니그램 정밀도 (Unigram Precision)

- 한국어-영어 번역기의 성능을 측정한다고 가정해보자.
- 두 개의 기계 번역기가 존재하고 두 기계 번역기에 같은 한국어 문장을 입력하여 번역된 영어 문장의 성능을 측정하고자 한다.
- 번역된 문장을 각각 Candidate1, 2라고 해보자.
- 이 문장의 성능을 평가하기 위해서는 정답으로 비교되는 문장이 있어야 한다.
- 세 명의 사람에게 한국어를 보고 영작해보라고 하여 세 개의 번역 문장을 만들어냈다.
- 이 세 문장을 각각 Reference1, 2, 3이라고 해보자.

**Example 1**

- Candidate1 : It is a guide to action which ensures that the military always obeys the commands of the party.  
- Candidate2 : It is to insure the troops forever hearing the activity guidebook that party direct.  
- Reference1 : It is a guide to action that ensures that the military will forever heed Party commands.  
- Reference2 : It is the guiding principle which guarantees the military forces always being under the command of the Party.  
- Reference3 : It is the practical guide for the army always to heed the directions of the party.

- 편의상 Candidate를 Ca로, Reference를 Ref로 축약하여 부르겠다.

<br>

- 이제 Ca 1, 2를 Ref 1, 2, 3과 비교하여 성능을 측정하고자 한다.
- 가장 직관적인 성능 평가 방법은 **Ref 1, 2, 3 중 어느 한 문장이라도 등장한 단어의 개수를 Ca에서 세는 것**이다.
- 그리고 그 후에 Ca의 모든 단어 카운트의 합, 즉 Ca에서의 총 단어의 수로 나눠준다.

- 이러한 측정 방법을 **유니그램 정밀도(Unigram Precision)**라고 한다.
- 이를 식으로 표현하면 다음과 같다.

$
\qquad
\text{Unigram Precision} 
= \frac{\text{Ref들 중에서 존재하는 Ca의 단어의 수}}{\text{Ca의 총 단어 수}} 
= \frac{\text{the number of Ca words(unigrams) which occur in any Ref}}{\text{the total number of words in the Ca}}
$

- Ca1의 단어들은 얼추 훑어만봐도 Ref1, Ref2, Ref3에서 전반적으로 등장하는 반면, Ca2는 그렇지 않다.
- 이는 Ca1이 Ca2보다 더 좋은 번역 문장임을 의미한다. (대소문자 구분은 없다고 가정)
  - Ca1의 It is a guide to action은 Ref1에서 등장
  - Ca1의 which는 Ref2에서 등장
  - Ca1의 ensures that the militrary는 Ref1에서 등장
  - Ca1의 always는 Ref2와 Ref3에서 등장
  - Ca1의 commands는 Ref1에서 등장
  - Ca1의 of the party는 Ref2에서 등장
  - Ca1에 있는 단어 중 Ref1, Ref2, Ref3 어디에도 등장하지 않은 단어는 obeys뿐이다.
- 반면 Ca2는 Ca1과 비교하여 상대적으로 Ref1, 2, 3에 등장한 단어들이 적다.

<br>

- 위의 계산 방법에 따르면 Ca1과 Ca2의 유니그램 정밀도는 각각 아래와 같다.

$
\qquad
\text{Ca1 Unigram Percision} = \frac{17}{18}
$

$
\qquad
\text{Ca2 Unigram Percision} = \frac{8}{14}
$

- 이제부터는 단어라는 표현보다는 유니그램이라는 용어로 설명한다.
- 지금까지 설명한 유니그램 정밀도는 나름 의미있는 측정 방법으로 보이지만 사실 허술한 점이 있다.
- 아래와 같이 새로운 예가 있다고 해보자.

<br>

### 3.1.2 보정된 유니그램 정밀도 (Modified Unigram Precision)

**Example 2**

- Candidate : the the the the the the the
- Reference1 : the cat is on the mat
- Reference2 : there is a cat on the mat

- 위의 Ca는 the만 7개가 등장한 터무니 없는 번역이다.
- 하지만 이 번역은 앞서 배운 유니그램 정밀도에 따르면 $\frac{7}{7} = 1$이라는 최고의 성능 평가를 받게 된다.
- 이에 유니그램 정밀도를 다소 보정할 필요를 느낀다.

<br>

- 이를 보정하기 위해서는 정밀도의 분자를 계산하기 위해 Ref와 매칭하며 카운트하는 과정에서 Ca의 유니그램이 이미 Ref에서 매칭된 적이 있었는 지를 고려해야 한다.

$
\qquad
\text{Unigram Precision} = \frac{\text{Ref들과 Ca를 고려한 새로운 카운트 방법이 필요!}}{\text{Ca의 총 유니그램 수}}
$

<br>

- 정밀도의 분자를 계산하기 위한 각 유니그램의 카운트는 다음과 같이 수정한다.
- 유니그램이 하나의 Ref에서 최대 몇 번 등장했는 지를 카운트한다.
- 이 값을 maximum reference count를 줄인 의미에서 Max_Ref_Count라고 부른다.
- Max_Ref_Count가 기존의 단순 카운트한 값보다 작은 경우에는 이 값을 최종 카운트 값으로 대체한다.
- 정밀도의 분자 계산을 위한 새로운 카운트 방식을  식으로 표현하면 다음과 같다.

$
\qquad
Count_{clip} = min(Count, \; Max\_Ref\_Count)
$

- 위의 카운트를 사용하여 분자를 계산한 정밀도를 **보정된 유니그램 정밀도(Modified Unigram Precision)**라고 한다.

$
\qquad
\text{Modified Unigram Precision} 
= \frac{\text{Ca의 각 유니그램에 대해 }Count_{clip} \text{을 수행한 값의 총 합}}{\text{Ca의 총 유니그램 수}}
= \frac{\sum_{unigram \in Candidate} Count_{clip} (unigram)}{\sum_{unigram \in Candidate} Count (unigram)}
$

- 분모의 경우에는 이전과 동일하게 Ca의 모든 유니그램에 대해서 각각 $Count$하고 모두 합한 값을 사용한다.

<br>

- 보정된 유니그램 정밀도를 예제를 통해 이해해보자.
- **Example 2**를 살펴보면
  - the의 경우에는 Ref1에서 총 두 번 등장 $\rightarrow$ the의 카운트는 2로 보정된다.
  - Ca의 기존 유니그램 정밀도는 $\frac{7}{7} = 1$이다.
  - 보정된 유니그램 정밀도는 $\frac{2}{7}$와 같이 변경된다.

<br>

- 다른 예로 **Example 1**에서의 Ca1의 보정된 유니그램 정밀도를 계산해보면 보정되기 이전과 동일하게 $\frac{17}{18}$이지만 결과를 얻는 과정은 다르다.
  - Ca1에서 the는 3번 등장하지만, Ref2와 Ref3에서 the가 4번 등장하므로 3이 4보다 작으므로 the는 3으로 카운트된다.
  - the 외에 Ca1의 모든 유니그램은 전부 1개씩 등장하므로 보정 전과 동일하게 카운트하면 된다.
  - 결과적으로 보정 이전의 정밀도와 동일하게 $\frac{17}{18}$의 값을 가진다.

<br>

### 3.1.3 보정된 유니그램 정밀도(Modified Unigram Precision) 구현하기

